In [1]:
!apt-get install -y openslide-tools > /dev/null 2>&1
!pip install openslide-python > /dev/null 2>&1

In [2]:
import numpy as np
import pandas as pd

import openslide
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import random
import os
import tensorflow as tf


E0000 00:00:1723285805.678129      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0810 10:30:05.686955669      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0810 10:30:05.686972366      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0810 10:30:05.686976223      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0810 10:30:05.686979132      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0810 10:30:05.686981976      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# EXTRACT PATCHES

In [4]:
def percentage(mask):
    return (np.sum(mask > 0) / mask.size) * 100
def extract_patches(im_slide, ms_slide, level, size_mask , num_patches_needed):
    f = int(ms_slide.level_downsamples[level])
    size_scale = im_slide.level_dimensions[level][0] // ms_slide.level_dimensions[level][0]
    coord_scale = im_slide.level_dimensions[0][0] // ms_slide.level_dimensions[level][0]
    size_image = (size_mask[0] * size_scale, size_mask[1] * size_scale)
    
    ms_width, ms_height = ms_slide.level_dimensions[level]
    
    image_patches , l= [], []
    for y_ms in range(0, ms_height, size_mask[1]):
        for x_ms in range(0, ms_width, size_mask[0]):
            l.append([x_ms , y_ms])
    
    count,used_indices =0, []
    random.seed(42)

    while count < num_patches_needed:
        index = random.randint(0, len(l) - 1)
        if index not in used_indices:
            used_indices.append(index)
            x_ms, y_ms = l[index][0] , l[index][1]
            x_im, y_im = x_ms * coord_scale, y_ms * coord_scale
            mask_patch = ms_slide.read_region((x_ms * f, y_ms * f), level, size_mask).convert("L")
            image_patch = im_slide.read_region((x_im, y_im), level, size_image).convert("RGB")
   
            if (percentage(np.array(mask_patch))) > 50  and (percentage(np.array(mask_patch))) > 50:
                image_patches.append(np.array(image_patch))
                count+=1
                if count == num_patches_needed:
                    break
        else:
            continue
    return  image_patches
# a=extract_patches(openslide.OpenSlide("/kaggle/input/dddddddd/images/case_radboud_0002.tif"), openslide.OpenSlide("/kaggle/input/dddddddd/masks/case_radboud_0002_tissue.tif"), 1, (64,64) , 5)
# print(np.shape(a))


# DATASET

In [5]:
class PatchDataset(tf.keras.utils.Sequence):
    def __init__(self, images_dir, masks_dir, csv_file, num_patches_per_image, level, size_mask, batch_size):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.df = pd.read_csv(csv_file)
        self.num_patches_per_image = num_patches_per_image
        self.level = level
        self.size_mask = size_mask
        self.batch_size = batch_size
        self.image_list = sorted(os.listdir(images_dir))
        
    def __len__(self):
        return len(self.image_list) // self.batch_size
    
    def __getitem__(self, idx):
        batch_images = []
        batch_labels = []
        
        for i in range(self.batch_size):
            image_file = self.image_list[idx * self.batch_size + i]
            impath = os.path.join(self.images_dir, image_file)
            mspath = os.path.join(self.masks_dir, image_file.replace('.tif', '_tissue.tif'))
            
            im_slide = openslide.OpenSlide(impath)
            ms_slide = openslide.OpenSlide(mspath)
            
            image_patches = extract_patches(im_slide, ms_slide, self.level, self.size_mask, self.num_patches_per_image)
            image_patches = np.array(image_patches, dtype=np.float32) 
            case_id_to_find = image_file
            filtered_row = self.df.loc[self.df['case_id'] == case_id_to_find].iloc[0]
            event = filtered_row["event"]
            years = filtered_row["follow_up_years"]
            
            image_labels = [[event, years]] * self.num_patches_per_image
            
            batch_images.append(image_patches)
            batch_labels.append(image_labels)
        
        return np.array(batch_images), np.array(batch_labels)
    def repeat(self):
        return self  # This allows the dataset to be used repeatedly



In [6]:
# images_dir = "/kaggle/input/dddddddd/images"
# masks_dir = "/kaggle/input/dddddddd/masks"
# csv_file = "/kaggle/input/dddddddd/training_labels.csv"

# dataset = PatchDataset(images_dir, masks_dir, csv_file, num_patches_per_image=6, level=1, size_mask=(64, 64) ,batch_size= 11)
# for a in dataset:
#     x,y=a
#     print(x.shape , y.shape)
    

# MODEL

In [7]:
class DoubleConv(layers.Layer):
    def __init__(self, filters):
        super(DoubleConv, self).__init__()
        self.conv1 = layers.Conv2D(filters, 3, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu = layers.ReLU()

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x

class UNet(Model):
    def __init__(self, num_patches , input_shape):
        super(UNet, self).__init__()
        self.num_patches = num_patches
        self.input_shape = input_shape

        self.inc = DoubleConv(64)
        self.down1 = self.down_block(128)
        self.down2 = self.down_block(256)
        self.down3 = self.down_block(512)
        self.down4 = self.down_block(1024)

        self.up1 = self.up_block(512)
        self.up2 = self.up_block(256)
        self.up3 = self.up_block(128)
        self.up4 = self.up_block(64)

        self.gap = layers.GlobalAveragePooling2D()
        self.fc1 = layers.Dense(32, activation='relu')
        self.fc2 = layers.Dense(2)  # 2 outputs: event and years

    def down_block(self, filters):
        return tf.keras.Sequential([
            layers.MaxPooling2D(2),
            DoubleConv(filters)
        ])

    def up_block(self, filters):
        return tf.keras.Sequential([
            layers.Conv2DTranspose(filters, 2, strides=2, padding='same'),
            DoubleConv(filters)
        ])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        x = tf.reshape(inputs, [-1, self.input_shape[0], self.input_shape[1], self.input_shape[2]])

        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5)
        x = tf.concat([x4, x], axis=-1)
        x = self.up2(x)
        x = tf.concat([x3, x], axis=-1)
        x = self.up3(x)
        x = tf.concat([x2, x], axis=-1)
        x = self.up4(x)
        x = tf.concat([x1, x], axis=-1)

        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)

        x = tf.reshape(x, [batch_size, self.num_patches, 2])

        return x
    


In [8]:
# model = UNet(num_patches=6 , input_shape=(512,512,3))
# sample_input = tf.random.normal([2, 6, 512,512, 3])
# output = model(sample_input)
# print(output.shape)  

# TPU SETUP

In [9]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)


I0000 00:00:1723285817.538762      77 service.cc:145] XLA service 0x5baaad8a7e10 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723285817.538814      77 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1723285817.538819      77 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1723285817.538822      77 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1723285817.538825      77 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1723285817.538828      77 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1723285817.538831      77 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1723285817.538833      77 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1723285817.538836      77 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


In [10]:
def epochnum(file):
    try:
        epoch_part = file.split('epoch')[1].split('-')[0]
        return int(epoch_part)
    except (IndexError, ValueError):
        return -1
    
    
    
def custom_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    
    years_loss = mse(y_true[:, :, 1], y_pred[:, :, 1])
    event_loss = mse(y_true[:, :, 0], y_pred[:, :, 0])

    res,a=(1 * event_loss ), (1* years_loss)

    return  res+a

# print(custom_loss(output, y))

In [11]:
print(len(os.listdir("/kaggle/input/dddddddd/images")))

55


In [12]:
            
# BATCH_SIZE = 1
# NUM_PATCHES = 4
# INPUT_SHAPE = (512, 512, 3)
# model = UNet(num_patches=NUM_PATCHES, input_shape=INPUT_SHAPE)
# latest_checkpoint = "/kaggle/working/epoch151-loss3.7436.weights.h5"


# sample_input = tf.zeros((1, NUM_PATCHES, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
# o = model(sample_input)
# model.load_weights(latest_checkpoint)

In [13]:
# im="/kaggle/input/dddddddd/images/case_radboud_0217.tif"
# ms= "/kaggle/input/dddddddd/masks/case_radboud_0217_tissue.tif"
# print(o.shape)
# print(sample_input.shape)

In [14]:
# a=extract_patches(openslide.OpenSlide(im), openslide.OpenSlide(ms), 1, (64,64) , 4)
# a = np.expand_dims(a, axis=0)  

# a_tensor = tf.convert_to_tensor(a, dtype=tf.float32)

# # print(type(a_tensor))
# # print(a_tensor.shape)

# out = model(a_tensor)
# print(out.shape)
# print(out)

In [15]:
# print(custom_loss(out, o))
# print(custom_loss(out, out))
# print(type(print(custom_loss(out, o))))

In [16]:
# print(out)

In [17]:
print(f"Number of TPU cores: {strategy.num_replicas_in_sync}")

Number of TPU cores: 8


# TRAINING 

In [ ]:
h5_files = [f for f in os.listdir("/kaggle/working") if f.endswith('.h5')]
resume=0
if h5_files:
    h5_files.sort(key=lambda x: epochnum(x), reverse=True)
    num=epochnum(h5_files[0])
    if num:
        resume=num
print(resume)

from tensorflow.keras.callbacks import ReduceLROnPlateau


BATCH_SIZE = 16
NUM_PATCHES = 4
INPUT_SHAPE = (512, 512, 3)
TOTAL_IMAGES = 55
STEPS_PER_EPOCH =  (TOTAL_IMAGES + BATCH_SIZE - 1) // BATCH_SIZE


numepochs=2002
epochs=resume+numepochs


images_dir = "/kaggle/input/dddddddd/images"
masks_dir = "/kaggle/input/dddddddd/masks"
csv_file = "/kaggle/input/dddddddd/training_labels.csv"
checkpoint_dir = "/kaggle/working"

# def lr_schedule(epoch, lr):
#     initial_lr = 1e-3
#     decay_rate = 0.9
#     decay_steps = 30
#     if epoch % decay_steps == 0 and epoch > 0:
#         lr = lr * decay_rate
#     return lr

# # Define the LearningRateScheduler callback
# lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.9, patience=3,verbose=True)


class CustomCheckpoint(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 200 == 0:
            self.model.save_weights(f"{checkpoint_dir}/epoch{epoch+1:03d}-loss{logs['loss']:.4f}.weights.h5")
            print(f" Model saved at epoch {epoch + 1}")

class CustomLogger(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch%50==0:
            print(f" Completed Epoch {epoch + 1}, Loss: {logs['loss']:.4f}")


def train_model(start_epoch=0, total_epochs=2):
    dataset = PatchDataset(images_dir, masks_dir, csv_file, num_patches_per_image=NUM_PATCHES, level=1, size_mask=(64, 64), batch_size=BATCH_SIZE)
    
    with strategy.scope():
        model = UNet(num_patches=NUM_PATCHES, input_shape=INPUT_SHAPE)
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-03)
        


        
        h5_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.h5')]
        if h5_files:
            h5_files.sort(key=lambda x: epochnum(x), reverse=True)
            latest_checkpoint = os.path.join(checkpoint_dir, h5_files[0])
            print(f"Resuming from checkpoint: {h5_files[0]}")
            try:
                sample_input = tf.zeros((1, NUM_PATCHES, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
                nothing= model(sample_input)
                model.load_weights(latest_checkpoint)
                model.compile(optimizer=optimizer, loss=custom_loss)
                total_params = model.count_params()
                print(f"Total number of parameters: {total_params:,}")

                start_epoch = epochnum(h5_files[0])
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Starting from scratch.")
        else:
            print("No checkpoint found. Starting from scratch.")        
        model.compile(optimizer=optimizer, loss=custom_loss)
        history = model.fit(
            dataset,
            initial_epoch=start_epoch,
            epochs=total_epochs,
            steps_per_epoch=STEPS_PER_EPOCH,
            verbose=1,
            callbacks=[CustomCheckpoint(), CustomLogger(), reduce_lr]
        )
    
    return history, model

try:
    history, trained_model = train_model(total_epochs=epochs)
    print("Training complete.")
except Exception as e:
    print(f"An error occurred during training: {e}")

In [19]:
import os
path = "/kaggle/working"
pth_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.h5')]
pth_files.sort(key=lambda x: epochnum(x) , reverse=True)
# for file in pth_files[1:]:
#     os.remove(os.path.join(path , file))
#     print(f"removed {file}")
# print(os.listdir(path))
# print(pth_files)


In [20]:
# os.remove(f"{path}/state.db")

In [21]:
# print(os.listdir(path))